In [1]:
import random
import torch

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
from warpctc_pytorch import CTCLoss
import os
import utils
import dataset
import crnn

In [2]:
dset = dataset.Dataset(img_root='/home/sherlock/Documents/express_recognition/data/train/telephone/', 
                       txt_root='/home/sherlock/Documents/express_recognition/data/train/telephone_label_train.txt')
dataloader = DataLoader(dset, batch_size=32, num_workers=4, collate_fn=dataset.alignCollate())

In [3]:
nh = 100
alphabet = '0123456789'
nclass = len(alphabet) + 1
nc = 3

converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

mynet = crnn.CRNN(32, nc, nclass, nh)

mynet.cuda()

criterion = criterion.cuda()

loss_avg = utils.averager()
optimizer = optim.Adadelta(mynet.parameters(), lr=1e-3)

def trainBatch(crnn, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    image = torch.FloatTensor(cpu_images)
    image = Variable(image).cuda()
    t, le = converter.encode(cpu_texts)
    text = torch.IntTensor(t)
    text = Variable(text)
    length = torch.IntTensor(le)
    length = Variable(length)
    preds = crnn(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    crnn.zero_grad()
    cost.backward()
    optimizer.step()
    return cost


for epoch in range(800):
    train_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        for p in mynet.parameters():
            p.requires_grad = True
        mynet.train()

        cost = trainBatch(mynet, criterion, optimizer)
        loss_avg.add(cost)
        i += 1
        if epoch % 100 == 0:
            if i % 10 == 0:
                print('[%d/%d][%d/%d] Loss: %f' %
                      (epoch, 10000, i, len(dataloader), loss_avg.val()))
                loss_avg.reset()

#         if i % opt.valInterval == 0:
#             val(crnn, test_dataset, criterion)

        # do checkpointing
#         if i % opt.saveInterval == 0:
#             torch.save(
#                 crnn.state_dict(),
#                 '{0}/netCRNN_{1}_{2}.pth'.format(opt.experiment, epoch, i))


[0/10000][10/52] Loss: 50.982706
[0/10000][20/52] Loss: 50.685906
[0/10000][30/52] Loss: 50.558454
[0/10000][40/52] Loss: 50.412920
[0/10000][50/52] Loss: 50.241802
[100/10000][10/52] Loss: 23.985806
[100/10000][20/52] Loss: 20.593498
[100/10000][30/52] Loss: 20.658182
[100/10000][40/52] Loss: 20.924697
[100/10000][50/52] Loss: 20.853966
[200/10000][10/52] Loss: 17.678644
[200/10000][20/52] Loss: 14.101414
[200/10000][30/52] Loss: 14.380521
[200/10000][40/52] Loss: 14.861000
[200/10000][50/52] Loss: 15.005163
[300/10000][10/52] Loss: 11.621246
[300/10000][20/52] Loss: 8.384962
[300/10000][30/52] Loss: 8.697979
[300/10000][40/52] Loss: 9.402159
[300/10000][50/52] Loss: 9.536990
[400/10000][10/52] Loss: 6.532733
[400/10000][20/52] Loss: 3.923311
[400/10000][30/52] Loss: 4.177174
[400/10000][40/52] Loss: 4.857419
[400/10000][50/52] Loss: 4.944170
[500/10000][10/52] Loss: 2.809282
[500/10000][20/52] Loss: 1.344193
[500/10000][30/52] Loss: 1.530953
[500/10000][40/52] Loss: 1.839359
[500/100

In [ ]:
nh = 100
alphabet = '0123456789'
nclass = len(alphabet) + 1
nc = 1
batch_size = 1

converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

mynet = crnn.CRNN(32, nc, nclass, nh)

mynet.cuda()

loss_avg = utils.averager()
optimizer = optim.Adadelta(mynet.parameters(), lr=1e-3)

In [ ]:
for epoch in range(10000):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 1):
        img, label = data
        img = Variable(img).cuda()
        t, le = converter.encode(label)
        text = torch.IntTensor(t)
        text = Variable(text)
        length = torch.IntTensor(le)
        length = Variable(length)
        preds = mynet(img)
        preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
        loss = criterion(preds, text, preds_size, length)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss
        if i % 100 == 0:
            print('{}, Loss: {}'.format(epoch, running_loss / (i * batch_size)))
    print()
print('Finish Training!')

In [7]:
import PIL.Image as Image
img_path = '/home/sherlock/Documents/express_recognition/data/train/telephone/290.png'
transformer = dataset.resizeNormalize((128, 32))
image = Image.open(img_path).convert('RGB')
image = transformer(image).cuda()
image = image.view(1, *image.size())
image = Variable(image)

mynet.eval()
preds = mynet(image)
_, preds = preds.max(2)
preds = preds.squeeze(2)
preds = preds.transpose(1, 0).contiguous().view(-1)
preds_size = Variable(torch.IntTensor([preds.size(0)]))
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
print('%-20s => %-20s' % (raw_pred, sim_pred))

118883399922---003333-3377111155- => 18392033715         
